Welcome to the School Achievement Dashboard! Here, we'll look at various factors to educate decision-makers on how best to invest to improve student learning outcomes.

*Produced by Colin Howard and Luke Moffit for CSCI 77800 -- EthiCS, Fall 2024*

# Part 1. Poverty and Achievement

Start by running all of the cells in this notebook. Then use the dropdown below to choose whether to look at city-, state-, or national-level data.

In [16]:
#region
# @title Level selector

from ipywidgets import *
import pandas as pd
import urllib.request as rq
import plotly.express as px
import plotly.graph_objects as go
import IPython.display as display
import json

levels = {
    "New York City": "XN",
    "New York State": "NY",
    "United States": "NT"
}

level = "NT"

def set_level(choice):
    level = choice

level_selector = Dropdown(options=levels,description="Level:")
interact(set_level,choice=level_selector);

#endregion

interactive(children=(Dropdown(description='Level:', options={'New York City': 'XN', 'New York State': 'NY', '…

In [17]:
subjects = ["Mathematics","Reading","Science"]

subject = "Mathematics"

subscales = {
    "Mathematics" : "MRPCM",
    "Reading": "RRPCM",
    "Science": "SRPUV"
}

subscale = "MRPCM"

years = {
    "Mathematics": [1990, 1992, 1996, 1996, 2000, 2000, 2003, 2005, 2007, 2009, 2011, 2013, 2015, 2017, 2019, 2022][::-1],
    "Reading": [1992, 1994, 1998, 1998, 2002, 2003, 2005, 2007, 2009, 2011, 2013, 2015, 2017, 2019, 2022][::-1],
    "Science": [2009, 2015, 2019][::-1]
}

year = 2019

grades = {
    "Mathematics": [4,8],
    "Reading": [4,8,12],
    "Science": [4,8,12]
}

grade = 8

variable = "SLUNCH3"

stattype = "MN:MN,PC:P1,PC:P2,PC:P5,PC:P7,PC:P9"

stattypes = {
    "MN:MN": "Mean",
    "PC:P1": "10th Percentile",
    "PC:P2": "25th Percentile",
    "PC:P5": "50th Percentile",
    "PC:P7": "75th Percentile",
    "PC:P9": "90th Percentile"
}

marker_colors = {
    "MN:MN": px.colors.qualitative.Prism[10],
    "PC:P1": px.colors.qualitative.Prism[7],
    "PC:P2": px.colors.qualitative.Prism[5],
    "PC:P5": px.colors.qualitative.Prism[4],
    "PC:P7": px.colors.qualitative.Prism[1],
    "PC:P9": px.colors.qualitative.Prism[0]
}

def build_frame(subject="Mathematics",year=2019):
    year = year
    subject = subject
    subscale = subscales[subject]
    response = rq.urlopen(f"https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data"
                        f"&subject={subject.lower()}"
                        f"&grade={grade}"
                        f"&subscale={subscale}"
                        f"&variable={variable}"
                        f"&jurisdiction={level}"
                        f"&stattype={stattype}"
                        f"&year={year}")
    response_string = response.read().decode('utf-8')
    try:
        df = pd.DataFrame(json.loads(response_string)['result'])
        return df
    except:
        return response_string

def build_figure(df):
    traces = []
    for stat in stattypes.keys():
        trace = go.Bar(
            x=df.loc[df['stattype']==stat]["varValueLabel"], 
            y=df.loc[df['stattype']==stat]["value"], 
            name=stattypes[stat], 
            marker={"color": marker_colors[stat]}
        )
        traces.append(trace)
    return traces

df = build_frame(subject,year)
fw = go.FigureWidget(data=build_figure(df),layout=go.Layout(barmode="group",title=f"{grade}th Grade {subject} NAEP Scores and Federal Lunch Program Eligibility, {year}, {level}"))

def update_figure(change):
    subject = subject_widget.value
    year = year_widget.value
    df = build_frame(subject,year)
    with fw.batch_update():
        for i, stat in enumerate(stattypes):
            fw.data[i].x=df.loc[df['stattype']==stat]['varValueLabel']
            fw.data[i].y=df.loc[df['stattype']==stat]['value']
        fw.update_layout(title=f"{grade}th Grade {subject} NAEP Scores and Federal Lunch Program Eligibility, {year}, {level}")
    
subject_widget = Dropdown(options=subjects,description="Subject:")
year_widget = Dropdown(options=years['Mathematics'],description="Year:")

def update_years(*args):
    year_widget.options = years[subject_widget.value]
subject_widget.observe(update_years, 'value')

subject_widget.observe(update_figure, 'value')
year_widget.observe(update_figure, 'value')

container = VBox([HBox([subject_widget,year_widget]),fw])
container

# interact(update_figure,subject=subject_widget,year=year_widget);

# Part 2. Poverty and Funding

# Part 3. Factors Affecting Achievement